In [ ]:
!pip install ipywidgets
!pip install ultralytics==8.0.196
!pip install roboflow


In [ ]:
from ipywidgets import widgets, FileUpload, VBox, HBox, Button, IntText, Output
from IPython.display import display, Image
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="TrGTFaa2Bzk971xjm8AL")
project = rf.workspace("bhuv-7sdq9").project("kidney-ql5tq")
version = project.version(4)
dataset = version.download("yolov8")

In [ ]:
# Define functions
def train_model(epochs, data_yaml):
    !yolo task=segment mode=train model=yolov8s-seg.pt data={data_yaml} epochs={epochs} imgsz=800 plots=True

def on_train_model(b):
    with output_train:
        output_train.clear_output()
        epochs = epoch_input.value
        data_yaml = data_yaml_input.value
        train_model(epochs, data_yaml)
        display(Image(filename='/content/runs/segment/train/confusion_matrix.png', width=600))
        display(Image(filename='/content/runs/segment/train/results.png', width=600))
        display(Image(filename='/content/runs/segment/train/val_batch0_pred.jpg', width=600))

def on_file_upload(change):
    os.makedirs('/content/runs/segment/upload_output', exist_ok=True)
    custom_model = YOLO('/content/runs/segment/train/weights/best.pt')  # Load your custom-trained model
    for name, file_info in uploader.value.items():
        with open(name, 'wb') as f:
            f.write(file_info['content'])
        img_path = os.path.join('/content/', name)
        results = custom_model(img_path)
        for result in results:
            plt.figure(figsize=(10, 10))
            plt.imshow(result.plot())
            plt.axis('off')
            output_img_path = os.path.join('/content/runs/segment/upload_output/', name)
            plt.savefig(output_img_path)
            plt.close()

        # Display the output image
        output_img_path = os.path.join('/content/runs/segment/upload_output/', name)
        display(Image(filename=output_img_path, width=600))

def on_display_results(b):
    results_dir = '/content/runs/segment/upload_output'
    with output_detect:
        output_detect.clear_output()
        for image_file in os.listdir(results_dir):
            if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check if it's an image file
                image_path = os.path.join(results_dir, image_file)
                display(Image(filename=image_path, width=600))

In [ ]:
# Create GUI elements for training
epoch_input = IntText(value=100, description='Epochs:', disabled=False)
data_yaml_input = widgets.Text(value='', placeholder='Path to data.yaml', description='Data YAML:', disabled=False)
train_button = Button(description='Train Model')
train_button.on_click(on_train_model)
output_train = Output()

train_section = VBox([epoch_input, data_yaml_input, train_button, output_train])
display(train_section)


In [ ]:
# Create GUI elements for uploading, detecting, and displaying results
uploader = FileUpload(accept='.jpg, .jpeg, .png', multiple=True)
uploader.observe(on_file_upload, names='value')

display_button = Button(description='Display Results')
display_button.on_click(on_display_results)
output_detect = Output()

detect_section = VBox([uploader, display_button, output_detect])
display(detect_section)